In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### 1. Load Data

In [2]:
df = pd.concat([
    pd.read_csv('../data/On_Time_On_Time_Performance_2016_1.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_2.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_3.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_4.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_5.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_6.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_7.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_8.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_9.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_10.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_11.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2016_12.csv')
,pd.read_csv('../data/On_Time_On_Time_Performance_2017_1.csv')
])

/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (48,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (48,77,84,85) have mixed types. Specify dty

### 2. Initial Analysis of the data

In [3]:
# Proportion of delays
float(df[df['DepDelay'] > 0].shape[0])/df.shape[0]

0.3444744486149967

In [4]:
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2016,1,1,6,3,2016-01-06,AA,19805,AA,N4YBAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,1,1,7,4,2016-01-07,AA,19805,AA,N434AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,1,1,8,5,2016-01-08,AA,19805,AA,N541AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,1,1,9,6,2016-01-09,AA,19805,AA,N489AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,1,1,10,7,2016-01-10,AA,19805,AA,N439AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.shape

(6067675, 110)

### 3. Transform the data

In [6]:
# Remove cancelled flights
df = df[df['Cancelled']<1] # The flight is cancelled

In [7]:
df.shape

(5992928, 110)

In [8]:
# Remove diverted flights
df = df[df['Diverted']<1] # The flight is diverted

In [9]:
df.shape

(5977790, 110)

In [10]:
###### Create a field for logistic regression (Classic method)

df['Class'] = 'No Delay'
df['Class'][(df['DepDelay'] >= 15) & (df['DepDelay'] < 30)] = 'Minor Delay'
df['Class'][(df['DepDelay'] >= 30) & (df['DepDelay'] < 60)] = 'Delay'
df['Class'][(df['DepDelay'] >= 60) & (df['DepDelay'] < 120)] = 'Significant Delay'
df['Class'][df['DepDelay'] > 120] = 'Severe Delay'


/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/_

In [11]:
# Check rows per class
df.groupby(['Class']).count()/len(df)

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
Class,,,,,,,,,,,,,,,,,,,,,
Delay,0.052730,0.052730,0.052730,0.052730,0.052730,0.052730,0.052730,0.052730,0.052730,0.052730,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Minor Delay,0.064045,0.064045,0.064045,0.064045,0.064045,0.064045,0.064045,0.064045,0.064045,0.064045,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
No Delay,0.826298,0.826298,0.826298,0.826298,0.826298,0.826298,0.826298,0.826298,0.826298,0.826298,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Severe Delay,0.021223,0.021223,0.021223,0.021223,0.021223,0.021223,0.021223,0.021223,0.021223,0.021223,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Significant Delay,0.035704,0.035704,0.035704,0.035704,0.035704,0.035704,0.035704,0.035704,0.035704,0.035704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
###### Create a field for logistic regression (Tensorflow)

df['ClassNo'] = 0
df['ClassNo'][(df['DepDelay'] >= 15) & (df['DepDelay'] < 30)] = 1
df['ClassNo'][(df['DepDelay'] >= 30) & (df['DepDelay'] < 60)] = 2
df['ClassNo'][(df['DepDelay'] >= 60) & (df['DepDelay'] < 120)] = 3
df['ClassNo'][df['DepDelay'] > 120] = 4

/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/am186122/.virtualenvs/TensorFlow/lib/python2.7/site-packages/ipykernel/_

In [13]:
# std_pop = df['DepDelay'].mean() + 3*df['DepDelay'].std() # Need to normalize the flight delay time. removing outliers.
# df.set_value(df[df['DepDelay'] < - std_pop].index.values,'DepDelay',- std_pop)
# df.set_value(df[df['DepDelay'] > std_pop].index.values,'DepDelay',std_pop)
# df['DepDelay'].max()
# df['DepDelay'].hist()

In [14]:
df.shape

(5977790, 112)

In [15]:
# Choose features
df_sub = df[['DayOfWeek', 'DayofMonth', 'Month', 'AirlineID', 'OriginAirportID', 'DestAirportID', 'CRSDepTime', 'Class']]

In [16]:
# Remove NAs
df_sub = df_sub.dropna()

In [17]:
df_sub.dtypes

DayOfWeek           int64
DayofMonth          int64
Month               int64
AirlineID           int64
OriginAirportID     int64
DestAirportID       int64
CRSDepTime          int64
Class              object
dtype: object

In [18]:
df_sub.shape

(5977790, 8)

In [20]:
# Change the data types
df_sub.DayOfWeek=df_sub.DayOfWeek.astype(str)
df_sub.DayofMonth=df_sub.DayofMonth.astype(str)
df_sub.Month=df_sub.Month.astype(str)
df_sub.OriginAirportID=df_sub.OriginAirportID.astype(str)
df_sub.DestAirportID=df_sub.DestAirportID.astype(str)
df_sub.AirlineID=df_sub.AirlineID.astype(str)

In [21]:
df_sub.dtypes

DayOfWeek          object
DayofMonth         object
Month              object
AirlineID          object
OriginAirportID    object
DestAirportID      object
CRSDepTime          int64
Class              object
dtype: object

In [22]:
# df_sub['DepDelay'].hist()
# df_sub['DepDelay'].median()
# df_sub.set_value(df_sub[df_sub['DepDelay'] < 0].index.values,'DepDelay',0.001)
# df_sub.set_value(df_sub[df_sub['DepDelay'] == 0].index.values,'DepDelay',0.001)
# df_sub['DepDelay'].hist()
# import math
# import numpy as np
# df_sub['DepDelayLog'] = df_sub['DepDelay'].apply(np.log)
# df_sub['DepDelay'].hist()
# from sklearn import preprocessing
# ss = preprocessing.StandardScaler()
# ss = ss.fit(df_sub['DepDelay'])
# ss.transform(df_sub['DepDelay']).mean()
# df_sub['DepDelayNorm'] = ss.transform(df_sub['DepDelay'])
# df_sub['DepDelayNorm'].hist()

In [23]:
# Change the time format
df_sub['CRSDepHH'] = df_sub.CRSDepTime/100

In [24]:
df_sub['CRSDepHH'] = df_sub['CRSDepHH'].astype(int)

In [25]:
df_sub['CRSDepHH'].tail()

450012     8
450013    14
450014    14
450015     7
450016    16
Name: CRSDepHH, dtype: int64

In [26]:
df_sub.CRSDepTime.tail()

450012     800
450013    1430
450014    1415
450015     755
450016    1620
Name: CRSDepTime, dtype: int64

In [27]:
df_sub.sample(10)

,DayOfWeek,DayofMonth,Month,AirlineID,OriginAirportID,DestAirportID,CRSDepTime,Class,CRSDepHH
157795,2,10,1,19790,13296,10397,545,No Delay,5
21128,1,9,5,19805,14107,12478,2155,No Delay,21
43937,3,6,1,19805,11057,12953,1830,No Delay,18
177935,5,30,9,19790,10397,14747,1954,Delay,19
376535,2,7,6,19790,14747,12889,800,No Delay,8
364407,5,13,1,19790,13931,10397,1615,No Delay,16
37134,7,28,2,20304,13487,11337,1022,No Delay,10
162976,3,14,12,19393,10423,10693,635,No Delay,6
141863,2,3,1,19930,14747,12523,729,No Delay,7
363995,7,4,9,20304,12129,13487,712,No Delay,7


### 4. Define models

In [28]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()

In [29]:
x, y = df_sub[['DayOfWeek', 
                'DayofMonth', 
               'Month', 
               'AirlineID', 
               'OriginAirportID', 
               'DestAirportID' ,
                'CRSDepHH']], df_sub['Class']
               

In [30]:
x.dtypes

DayOfWeek          object
DayofMonth         object
Month              object
AirlineID          object
OriginAirportID    object
DestAirportID      object
CRSDepHH            int64
dtype: object

In [31]:
x.sample()

,DayOfWeek,DayofMonth,Month,AirlineID,OriginAirportID,DestAirportID,CRSDepHH
184425,1,1,2,20409,10423,12954,15


In [32]:
y.dtypes

dtype('O')

In [34]:
y.value_counts() # We need to balance the classes

No Delay             4939438
Minor Delay           382848
Delay                 315208
Significant Delay     213431
Severe Delay          126865
Name: Class, dtype: int64

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [37]:
X_train.shape

(4184453, 7)

In [38]:
y_train.shape

(4184453,)

In [39]:
# Decision Tree fitting a model
clf = clf.fit(X_train, y_train)

In [40]:
from sklearn.metrics import precision_recall_fscore_support

In [41]:
# Check model accuracy
y_pred = clf.predict(X_test)
y_pred_train = clf.predict(X_train)

In [42]:
#Test
precision_recall_fscore_support(y_test, y_pred, average='macro')

(0.25671419598230677, 0.26438164294909672, 0.25999179908843362, None)

In [43]:
#Train
precision_recall_fscore_support(y_train, y_pred_train, average='macro')

(0.99586987879137356, 0.99612815454168402, 0.9959845617468126, None)

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.70814074543713756

In [45]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred_train)

0.99850087932640175

In [46]:
# Balance the classes
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=42)
# X_res, y_res = rus.fit_sample(x, y)
# y_res = pd.Series(y_res)
# X_res = pd.DataFrame(X_res,columns=x.columns)
# y_res.value_counts()

In [47]:
from sklearn.externals import joblib
filename = 'final_model.pkl'
joblib.dump(clf, filename)

['final_model.pkl']

### 5. Tensorflow

In [48]:
import tensorflow as tf

In [49]:
df_sub_tf = df[['DayOfWeek', 
                'DayofMonth', 
                'Month', 
                'AirlineID', 
                'OriginAirportID', 
                'DestAirportID', 
                'CRSDepTime', 
                'ClassNo']]

In [50]:
df_sub_tf.shape

(5977790, 8)

In [51]:
df_sub_tf = df_sub_tf.dropna()

In [52]:
df_sub_tf.dtypes

DayOfWeek          int64
DayofMonth         int64
Month              int64
AirlineID          int64
OriginAirportID    int64
DestAirportID      int64
CRSDepTime         int64
ClassNo            int64
dtype: object

In [53]:
df_sub_tf['CRSDepHH'] = df_sub_tf.CRSDepTime/100

In [54]:
df_sub_tf['CRSDepHH'] = df_sub_tf['CRSDepHH'].astype(int)

In [55]:
df_sub_tf.dtypes

DayOfWeek          int64
DayofMonth         int64
Month              int64
AirlineID          int64
OriginAirportID    int64
DestAirportID      int64
CRSDepTime         int64
ClassNo            int64
CRSDepHH           int64
dtype: object

In [56]:
x, y = df_sub_tf[['DayOfWeek', 
                'DayofMonth', 
               'Month', 
               'AirlineID', 
               'OriginAirportID', 
               'DestAirportID' ,
                'CRSDepHH']], df_sub_tf['ClassNo']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [58]:
X_train.dtypes

DayOfWeek          int64
DayofMonth         int64
Month              int64
AirlineID          int64
OriginAirportID    int64
DestAirportID      int64
CRSDepHH           int64
dtype: object

In [59]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)

In [80]:
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns, hidden_units=[10,20,10], n_classes=5)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1481cd390>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [81]:
from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection

import tensorflow as tf

In [82]:
# Fit and predict.
classifier.fit(X_train[:1000], y_train[:1000],steps=2000)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Sav

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x1481cdc90>, 'hidden_units': [10, 20, 10], 'feature_columns': (_RealValuedColumn(column_name='', dimension=7, default_value=None, dtype=tf.float64, normalizer=None),), 'embedding_lr_multipliers': None, 'optimizer': None, 'dropout': None, 'gradient_clip_norm': None, 'activation_fn': <function relu at 0x1261b19b0>, 'input_layer_min_slice_size': None})

In [64]:
predictions = list(classifier.predict(X_test, as_iterable=True))
score = metrics.accuracy_score(y_test, predictions)
print('Accuracy: {0:f}'.format(score))

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Accuracy: 0.825849
